# **Importación del conjunto de datos**

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles

# Creamos nuestros datos artificiales, donde buscaremos clasificar
# dos anillos concéntricos

X,Y = make_circles(n_samples=500, factor=0.5, noise=0.05)

# Resolución del mapa de predicción
res = 100

# Coordenadas del mapa de predicción
_x0 = np.linespace(-1.5,1.5,res)
_x1 = np.linespace(-1.5,1.5,res)

# Input con cada combo de coordenadas del mapa de predicción
_pX = np.array(np.meshgrid(_x0,_x1)).T.reshape(-1,2)

# Objeto vacío a 0.5 del mapa de predicción
_pY = np.zeros((res,res)) + 0.5

# Visualización del mapa de predicción
plt.figure(figsize=(8,8))
plt.pcolormesh(_x0, _x1, _pY, cmap="coolwarm", vmin=0, vmax=1)

# Visualización de la nube de datos
plt.scatter(x[Y==0,0], X[Y==0,1], c="skyblue")
plt.scatter(x[Y==1,0], X[Y==1,1], c="salmon")

plt.tick_params(labelbottom=False, labelleft=False)


# **Redes Neuronales con Tensor Flow**

In [ ]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from matplotlib import animation
from IPython.core.display import display, HTML

# Definimos los puntos de entrada de la red, para la matriz X e Y.
iX = tf.placeholder('float', shape = [None, X.shape[1]])
iX = tf.placeholder('float', shape = [None])

lr = 0.01          # learning rate
nn = [2, 16, 8, 1] # número de neuronas por capa

# Capa 1
W1 = tf.Variable(tf.random_normal([nn[0], nn[1]]), name = 'Weights_1')
b1 = tf.Variable(tf.random_normal([nn[1]]), name = 'bias_1')
l1 = tf.nn.relu(tf.add(tf.matmul(iX,W1),b1))

# Capa 2
W2 = tf.Variable(tf.random_normal([nn[1], nn[2]]), name = 'Weights_2')
b2 = tf.Variable(tf.random_normal([nn[2]]), name = 'bias_2')
l2 = tf.nn.relu(tf.add(tf.matmul(iX,W2),b2))

# Capa 3
W3 = tf.Variable(tf.random_normal([nn[2], nn[3]]), name = 'Weights_3')
b3 = tf.Variable(tf.random_normal([nn[3]]), name = 'bias_3')
l3 = tf.nn.relu(tf.add(tf.matmul(iX,W3),b3))

# Vector de predicciones de Y
pY = tf.nn.sigmoid(tf.add(tf.matmul(l2, W3), b3))[:,0]

# Evaluación de las predicciones
loss = tf.losses.mean_squared_error(pY, iY)

# Definimos al optimizador de la red, para que minimice el error
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)
n_steps = 1000 # Número de ciclos de entrenamiento.
iPY = [] #Aquí guardaremos la evolución de las predicciones, para la animación

with tf.Session() as sess:
  # Inicializamos todos los parámetro de la red, las matrices W y b.
  sess.run(tf.global_variables_initializer())

  # Iteramos n pases de entrenamiento.
  for step in range(n_steps):
    # Evaluamos al optimizador, a la función de coste y al tensor de salida pY.
    # La evaluación del optimizer producirá el entrenamiento de la red
    _, _loss, _pY = sess.run([optimizer, loss, pY], feed_dict = {iX : X, iY : Y})

    # Cada 25b iteraciones, imprimimos métricas.
    if step % 25 == 0:
      # Cálculo del accuracy
      acc = np.mean(np.round(_pY) == Y)
      # Impresión de métricas
      print('Step', step, '/', n_steps, '- Loss = ', _loss, ' -Acc = ', acc)
      # Obtenemos predicciones para cada punto de nuestro mapa de predicción _pX.
      _pY = sess.run(pY, feed_dict = {iX : _pX}).reshape((res,res))
      # Y lo guardamos para visualizar la animación
      iPY.append(_pY)

# --------------------------- CÓDIGO DE ANIMACIÓN -----------------------------#

ims = []
fig = plt.figure(figsize=(10,10))
print("---- Generando animación ----")

for fr in range(len(iPY)):
  im = plt.pcolormesh(_x0, _x1, iPY[fr], cmap='coolwarm', animated=True)
  # Visualización de la nube de datos
  plt.scatter(X[Y == 0,0], X[Y == 0,1], c = "skyblue")
  plt.scatter(X[Y == 1,0], X[Y == 1,1], c = "salmon")
  # plt.title('Resultado Clasificación')
  plt.tick_params(labelbottom=False, labelleft=False)
  ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit= True, repeat_delay= 1000)




# **Redes Neuronales con Keras**


In [ ]:
import tensorflow as tf
import tensorflow.keras as kr

from IPython.core.display import display, HTML

lr = 0.01          # Learning Rate
nn = [2, 16, 8, 1] # Número de neuronas por capa

# Creamos el objeto que contendrá a neustra red neuronal, como
# secuencia de capas

model = kr.Sequential()

# Añadimos la capa 1
l1 = model.add(kr.layers.Dense(nn[1], activation = 'relu'))

# Añadimos la capa 2
l2 = model.add(kr.layers.Dense(nn[2], activation = 'relu'))

# Añadimos la capa 3
l3 = model.add(kr.layers.Dense(nn[3], activation = 'sigmoid'))

# Compilamos el modelo, definiendo la función de coste y el optimizador
model.compile(loss='mse', optimizer = kr.optimizer.SGD(lr=0.05), metrics=['acc'])

# Y entrenamos al modelo. Los callbacks
model.fit(X,Y, epochs=100)


# **Redes Neuronales con Scikit Learn**

In [ ]:
import sklearn as sk
import sklearn.neual_network

from IPython.core.display import display, HTML

lr = 0.01          # Learning Rate
nn = [2, 16, 8, 1] # Número de neuronas por capa

# Creamos el objeto del modelo de red neuronal multicapa
clf = sk.neural_network.MLRegressor(solver = 'sgd',
                                    learning_rate_init = lr,
                                    hidden_layer_size=tuple(nn[1:]),
                                    verbose=True,
                                    n_iter_no_change=1000,
                                    batch_size = 64)

# Y lo entrenamos con nuestros datos
clf.fit(X,Y)